In [1]:
import numpy as np
import pandas as pd
from difflib import SequenceMatcher
import pickle

In [2]:
"""
input: filename
Pull data from pkl
output: data, teams
"""
def getData(name):
    daten= pd.read_pickle(name)
    data=daten[0]
    teams=daten[2]
    return data, teams


"""
input: team 1, team2
Search the right index in team 2 to insert them afterwards
team1= "Original index", team2= Next season
output: indexlist, team1
"""
def getIndex(team1, team2):
    
    indexlist=[]
    for mannschaft1 in team1:
        gefunden=0
        for mannschaft2 in team2:
            if mannschaft1== mannschaft2:
                #print(mannschaft2, team2.index(mannschaft1))
                index1= team1.index(mannschaft1)
                index2= team2.index(mannschaft1)
                indexlist.append([index1,index2])
                gefunden=1
                break
        #Absteiger mit -1 versehen
        if gefunden==0:
            index1= team1.index(mannschaft1)
            index2= -1
            indexlist.append([index1,index2])
    #Aufsteiger
    laenge= len(indexlist)
    aufsteiger=(list(set(team2)-set(team1)))
    for team in aufsteiger:
        indexlist.append([laenge,team2.index(team)])
        team1.append(team)
        laenge+=1
        
    return indexlist,team1

"""
input: index, number
Auxiliary method to replace the -1 for the promoted team.
Since the relegated team has a -1 for team2, must search the second indice and
give the first one back
ouput: inndex
"""
def get_index2(index,nummer):
    
    for i in index:
        if nummer ==i[1]:
            return i[0]

"""
input: data, index
Indicies of the playing teams must be changed
Teams from season1 and season2 have different indices for the same teams
Put these together now correctly
output: data
"""
def change_index(data,index):
    matrix= np.zeros((18,34,72))

    for spiel in data:
        mannschaften= spiel[-2:]
        mannschaften_neu=[]
        for m in mannschaften:
            for i in index:
                if m==i[1]:
                    if i[1] !=-1:
                        mannschaften_neu.append(i[0])
                    else:
                        mannschaften_neu.append(get_index2(index, i[0]))
                    break
        spiel[-2:]= mannschaften_neu
    #print(index)       
    return data
"""
input: season1, season2, index, total number of season
Put two seasons together now, but watch out for promoted and relegated teams
output: matrix
"""
def concat(data1, data2, index,n):

    matrix= np.zeros((len(index),34*n,72))
    #index= swap(index)
    for indices in index:
        #Normaler Case
        if indices[0] != 9 and indices[0] !=14 and indices[0]!=18 and indices[0]!=19:
            data1_neu=data1[indices[0]]
            data2_neu= (change_index(data2[indices[1]],index))
        #Absteiger
        elif indices[0] == 9 or indices[0] ==14:
            data1_neu=data1[indices[0]]
            data2_neu=np.array(34*[72*[-1]])
        #Aufsteiger
        elif indices[0]==18 or indices[0]==19:
            data1_neu=np.array(34*[72*[-1]])
            data2_neu= (change_index(data2[indices[1]],index))
        matrix[indices[0]]= np.concatenate((data1_neu,data2_neu), axis=0)    
    return matrix

"""
input: season1, season2, index, total number of season
Put season to concat matrix together now, but watch out for promoted and relegated teams
output: matrix
"""
def concat2(data1, data2, index,n):

    matrix= np.zeros((len(index),34*n,72))
    for indices in index:
        #print(indices)
        if indices[0] != 7 and indices[0] !=8:
            data1_neu=data1[indices[0]]
            data2_neu= (change_index(data2[indices[1]],index))
        elif indices[0] == 7 or indices[0] ==8:
            data1_neu=data1[indices[0]]
            data2_neu=np.array(34*[72*[-1]])
        matrix[indices[0]]= np.concatenate((data1_neu,data2_neu), axis=0)    
    return matrix

"""
input: matrix, team list
create pickle with data
output: pickle file
"""
def create_pkl(matrix,team):
    gesamt=[matrix,team]
    with open('Daten161718.pkl','wb') as f: 
        pickle.dump(gesamt, f)

In [2]:
"""
main to call all functions
"""
def main():
    
    #Preparation
    data16, team16= getData("Daten2016.pkl")
    data17, team17= getData("Daten2017.pkl")
    data18, team18= getData("Daten2018.pkl")
    
    #1.run: Create/modify index list, create team1617.
    # modify statistics from 17
    # statistics concat
    index1617, team1617= getIndex(team16, team17)#Indicies modifizieren
    matrix1=concat(data16,data17,index1617,2)
    
    #2.run: like in 1
    index161718, team161718= getIndex(team1617, team18)
    matrix2= concat2(matrix1, data18, index161718, 3)
    #3: pkl 
    create_pkl(matrix2,team161718)

#main()